In [5]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [10]:
def search(q):
    q = q.replace('\n', '').replace('（', '(').replace('）', ')')
    print("START : " + q)
    q = re.sub("\(.+\)$", "", " ".join(q.split()[:-3]), re.UNICODE)
    query = re.sub('(!|\u3000|/|\s|>|<|\.)+', "%20", q)
    url_search = 'https://eiga.com/search/' + query
    res_search = requests.get(url_search )
    res_search.encoding = res_search.apparent_encoding
    soup_search = BeautifulSoup(res_search.content, "lxml")
    result =  soup_search.find('section', attrs={"id": "rslt-movie"})
    if(result is not None):
        path = result.find('li', attrs={"class": "col-s-3"}).find('a')["href"]
        url_review = 'https://eiga.com' + path + 'review/all/'
        return url_review
    else:
        print("**************************************************")
        print(q + " HAS NO RESULT")
        print("**************************************************")
        return "error"

In [7]:
def scrape_review(query):
    page_num = 1
    data = {
        "id": -1,
        "reviews":{
            "eigacom":[],
            "filmarks":[],
            "coco":[],
        }
    }
    url_review=search(query)
    
    if(url_review == "error"):
        return "error"
    
    while(1):
        res = requests.get(url_review + str(page_num))
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.content, "lxml")
        if soup.find('div', attrs={"class": "user-review"}) == None: # ページ数の上限を超えたら
            print('DONE : ' + query )
            break
            
        for r in soup.find_all('div', attrs={"class": "user-review"}):
            title = r.find('h2',attrs={"class": "review-title"}).find('a')
            main_text =  r.find('div',attrs={"class": "txt-block"})
            tgl_btn = main_text.find('div',attrs={"class": "toggle-btn"})
            if tgl_btn is not None:
                tgl_btn.decompose()
            d = title.text + "\n" +  main_text.text.replace("\n", "")
            data["reviews"]["eigacom"].append(d)
        page_num += 1
        time.sleep(1)
    return data

In [8]:
input_file = '../2018_movie_clean'

In [9]:
num = 1
for q in open(input_file, 'r', encoding='utf-8').readlines():
    print(num)
    output_file = './{0}.json'.format(num)
    with open(output_file, 'w') as f:
        data = scrape_review(q)
        if(data == "error"):
            num += 1
            continue
        data["id"] = num
        jsn =  json.dumps(data,ensure_ascii=False, indent=2) 
        f.write(jsn)   
    f.close()
    num += 1

1
START : ZEN FOR NOTHING〜何でもない禅〜	2018	1	2
DONE : ZEN FOR NOTHING〜何でもない禅〜	2018	1	2

2
START : 映画 中二病でも恋がしたい！ -Take On Me-	2018	1	2
DONE : 映画 中二病でも恋がしたい！ -Take On Me-	2018	1	2

3
START : アンダー・ザ・ウォーター（ スウェーデン・ デンマーク・ フィンランド）	2018	1	5
**************************************************
アンダー・ザ・ウォーター（ スウェーデン・ デンマーク・ フィンランド） HAS NO RESULT
**************************************************
4
START : 嘘八百	2018	1	5
DONE : 嘘八百	2018	1	5

5
START : キングスマン: ゴールデン・サークル	2018	1	5
DONE : キングスマン: ゴールデン・サークル	2018	1	5

6
START : ジャコメッティ 最後の肖像	2018	1	5
DONE : ジャコメッティ 最後の肖像	2018	1	5

7
START : ナオト・インティライミ冒険記 旅歌ダイアリー2 後編	2018	1	5
DONE : ナオト・インティライミ冒険記 旅歌ダイアリー2 後編	2018	1	5

8
START : 愛の病	2018	1	6
DONE : 愛の病	2018	1	6

9
START : 牙狼<GARO> 神ノ牙 KAMINOKIBA	2018	1	6
DONE : 牙狼<GARO> 神ノ牙 KAMINOKIBA	2018	1	6

10
START : シークレット・デイ	2018	1	6
DONE : シークレット・デイ	2018	1	6

11
START : 68キル（英語版）	2018	1	6
DONE : 68キル（英語版）	2018	1	6

12
START : ジャッカルズ（英語版）	2018	1	6
DONE : ジャッカルズ（英語版）	2018	1	6

13
START : Z Inc. ゼット・インク（英語版）	2018	1	6


DONE : 空海-KU-KAI- 美しき王妃の謎	2018	2	24

114
START : さよならの朝に約束の花をかざろう	2018	2	24
DONE : さよならの朝に約束の花をかざろう	2018	2	24

115
START : あなたの旅立ち、綴ります	2018	2	24
DONE : あなたの旅立ち、綴ります	2018	2	24

116
START : 花は咲くか	2018	2	27
DONE : 花は咲くか	2018	2	27

117
START : シェイプ・オブ・ウォーター	2018	3	1
DONE : シェイプ・オブ・ウォーター	2018	3	1

118
START : 15時17分、パリ行き	2018	3	1
DONE : 15時17分、パリ行き	2018	3	1

119
START : ブラックパンサー	2018	3	1
DONE : ブラックパンサー	2018	3	1

120
START : ダウンサイズ	2018	3	2
DONE : ダウンサイズ	2018	3	2

121
START : アクト・オブ・バイオレンス	2018	3	3
DONE : アクト・オブ・バイオレンス	2018	3	3

122
START : おもてなし	2018	3	3
DONE : おもてなし	2018	3	3

123
START : 聖なる鹿殺し キリング・オブ・ア・セイクリッド・ディア	2018	3	3
DONE : 聖なる鹿殺し キリング・オブ・ア・セイクリッド・ディア	2018	3	3

124
START : ドラえもん のび太の宝島	2018	3	3
DONE : ドラえもん のび太の宝島	2018	3	3

125
START : ハッピーエンド	2018	3	3
DONE : ハッピーエンド	2018	3	3

126
START : ヒューマン・ハンター	2018	3	3
DONE : ヒューマン・ハンター	2018	3	3

127
START : プリンシパル 〜恋する私はヒロインですか？〜	2018	3	3
DONE : プリンシパル 〜恋する私はヒロインですか？〜	2018	3	3

128
START : しまじろう まほうのしまのだいぼうけん	2018	3	9
DONE : しまじろう まほうのしまのだい

DONE : METライブビューイング2017-18／ヴェルディ《ルイザ・ミラー》	2018	5	19

231
START : モリのいる場所	2018	5	19
DONE : モリのいる場所	2018	5	19

232
START : 犬ヶ島	2018	5	25
DONE : 犬ヶ島	2018	5	25

233
START : 宇宙戦艦ヤマト2202 愛の戦士たち/第五章 煉獄篇	2018	5	25
DONE : 宇宙戦艦ヤマト2202 愛の戦士たち/第五章 煉獄篇	2018	5	25

234
START : ゲティ家の身代金	2018	5	25
DONE : ゲティ家の身代金	2018	5	25

235
START : 恋は雨上がりのように	2018	5	25
DONE : 恋は雨上がりのように	2018	5	25

236
START : 妻よ薔薇のように 家族はつらいよ3	2018	5	25
**************************************************
妻よ薔薇のように 家族はつらいよ3 HAS NO RESULT
**************************************************
237
START : 友罪	2018	5	25
DONE : 友罪	2018	5	25

238
START : I AM THE BLUES アイ・アム・ザ・ブルース	2018	5	26
DONE : I AM THE BLUES アイ・アム・ザ・ブルース	2018	5	26

239
START : 兄友	2018	5	26
DONE : 兄友	2018	5	26

240
START : いつだってやめられる 10人の怒れる教授たち	2018	5	26
DONE : いつだってやめられる 10人の怒れる教授たち	2018	5	26

241
START : 海を駆ける	2018	5	26
DONE : 海を駆ける	2018	5	26

242
START : 男と女、モントーク岬で	2018	5	26
DONE : 男と女、モントーク岬で	2018	5	26

243
START : ガチ星	2018	5	26
DONE : ガチ星	2018	5	26

244
START : 軍中楽園 

DONE : KUICHISAN	2018	7	1

347
START : アーリーマン 〜ダグと仲間のキックオフ!〜	2018	7	6
DONE : アーリーマン 〜ダグと仲間のキックオフ!〜	2018	7	6

348
START : セラヴィ！	2018	7	6
DONE : セラヴィ！	2018	7	6

349
START : 虹色デイズ	2018	7	6
DONE : 虹色デイズ	2018	7	6

350
START : バトル・オブ・ザ・セクシーズ	2018	7	6
DONE : バトル・オブ・ザ・セクシーズ	2018	7	6

351
START : わがチーム、墜落事故からの復活	2018	7	6
DONE : わがチーム、墜落事故からの復活	2018	7	6

352
START : エヴァ	2018	7	7
DONE : エヴァ	2018	7	7

353
START : 菊とギロチン	2018	7	7
DONE : 菊とギロチン	2018	7	7

354
START : 君が君で君だ	2018	7	7
DONE : 君が君で君だ	2018	7	7

355
START : 熊野から イントゥ・ザ・新宮	2018	7	7
DONE : 熊野から イントゥ・ザ・新宮	2018	7	7

356
START : K SEVEN STORIES Episode 1 「R:b ～BLAZE～」	2018	7	7
DONE : K SEVEN STORIES Episode 1 「R:b ～BLAZE～」	2018	7	7

357
START : K SEVEN STORIES Spin-off Short Movie 「ザ・アイドルK」	2018	7	7
**************************************************
K SEVEN STORIES Spin-off Short Movie 「ザ・アイドルK」 HAS NO RESULT
**************************************************
358
START : 縄文にハマる人々	2018	7	7
DONE : 縄文にハマる人々	2018	7	7

359
START : スウィンダラーズ	2018	7	7


DONE : 詩季織々	2018	8	4

457
START : 東京ノワール	2018	8	4
DONE : 東京ノワール	2018	8	4

458
START : どうしようもない恋の唄	2018	8	4
DONE : どうしようもない恋の唄	2018	8	4

459
START : 7号室	2018	8	4
DONE : 7号室	2018	8	4

460
START : 2重螺旋の恋人	2018	8	4
DONE : 2重螺旋の恋人	2018	8	4

461
START : バンクシーを盗んだ男	2018	8	4
DONE : バンクシーを盗んだ男	2018	8	4

462
START : 触れたつもりで	2018	8	4
DONE : 触れたつもりで	2018	8	4

463
START : アローン	2018	8	6
DONE : アローン	2018	8	6

464
START : セクシャリティー	2018	8	7
DONE : セクシャリティー	2018	8	7

465
START : アルカディア	2018	8	8
DONE : アルカディア	2018	8	8

466
START : メイヘム 殺人晩餐会	2018	8	8
DONE : メイヘム 殺人晩餐会	2018	8	8

467
START : オーシャンズ8	2018	8	10
DONE : オーシャンズ8	2018	8	10

468
START : 追想	2018	8	10
DONE : 追想	2018	8	10

469
START : 英国総督 最後の家	2018	8	11
DONE : 英国総督 最後の家	2018	8	11

470
START : ブッシュウィック-武装都市-	2018	8	11
DONE : ブッシュウィック-武装都市-	2018	8	11

471
START : ゾンからのメッセージ	2018	8	11
DONE : ゾンからのメッセージ	2018	8	11

472
START : タイニー・ファニチャー	2018	8	11
DONE : タイニー・ファニチャー	2018	8	11

473
START : 歯まん	2018	8	11
DONE : 歯まん	2018	8	11

474
START : よるのたんけん	2018	8	1

DONE : だめんずうぉ～か～ パッション編	2018	9	28

579
START : 散り椿	2018	9	28
DONE : 散り椿	2018	9	28

580
START : DTC -湯けむり純情篇- from HiGH&LOW	2018	9	28
DONE : DTC -湯けむり純情篇- from HiGH&LOW	2018	9	28

581
START : 劇場版「フリクリ プログレ」	2018	9	28
DONE : 劇場版「フリクリ プログレ」	2018	9	28

582
START : 愛と法	2018	9	29
DONE : 愛と法	2018	9	29

583
START : 運命は踊る	2018	9	29
DONE : 運命は踊る	2018	9	29

584
START : ガールズ&パンツァー 第63回戦車道全国高校生大会 総集編	2018	9	29
DONE : ガールズ&パンツァー 第63回戦車道全国高校生大会 総集編	2018	9	29

585
START : 言葉のいらない愛	2018	9	29
**************************************************
言葉のいらない愛 HAS NO RESULT
**************************************************
586
START : 太陽の塔	2018	9	29
DONE : 太陽の塔	2018	9	29

587
START : 黙ってピアノを弾いてくれ	2018	9	29
DONE : 黙ってピアノを弾いてくれ	2018	9	29

588
START : 劇場版 夏目友人帳 〜うつせみに結ぶ〜	2018	9	29
DONE : 劇場版 夏目友人帳 〜うつせみに結ぶ〜	2018	9	29

589
START : BOURBON TALK	2018	9	29
**************************************************
BOURBON TALK HAS NO RESULT
**************************************************
590
START : BEATOPIA	2018	9	29
****

DONE : Workers 被災地に起つ	2018	10	20

684
START : あいあい傘	2018	10	26
DONE : あいあい傘	2018	10	26

685
START : 狼チャイルド	2018	10	26
DONE : 狼チャイルド	2018	10	26

686
START : オズランド 笑顔の魔法おしえます。	2018	10	26
DONE : オズランド 笑顔の魔法おしえます。	2018	10	26

687
START : 怪怪怪怪物! ( 台湾)	2018	10	26
**************************************************
怪怪怪怪物! ( 台湾) HAS NO RESULT
**************************************************
688
START : 黒蝶の秘密	2018	10	26
DONE : 黒蝶の秘密	2018	10	26

689
START : search/サーチ	2018	10	26
DONE : search/サーチ	2018	10	26

690
START : 栞	2018	10	26
DONE : 栞	2018	10	26

691
START : 旅猫リポート	2018	10	26
DONE : 旅猫リポート	2018	10	26

692
START : バーフバリ 伝説誕生 <完全版>	2018	10	26
DONE : バーフバリ 伝説誕生 <完全版>	2018	10	26

693
START : 嘘はフィクサーのはじまり	2018	10	27
DONE : 嘘はフィクサーのはじまり	2018	10	27

694
START : 岡本太郎の沖縄	2018	10	27
DONE : 岡本太郎の沖縄	2018	10	27

695
START : ガンジスに還る	2018	10	27
DONE : ガンジスに還る	2018	10	27

696
START : サクらんぼの恋	2018	10	27
DONE : サクらんぼの恋	2018	10	27

697
START : 終焉までの30秒	2018	10	27
DONE : 終焉までの30秒	2018	10	27

698
START : 13回

DONE : ボリショイ・バレエ in シネマ Season 2017-2018 「ジゼル」	2018	11	17

796
START : MAKI マキ	2018	11	17
DONE : MAKI マキ	2018	11	17

797
START : メサイア －幻夜乃刻－	2018	11	17
DONE : メサイア －幻夜乃刻－	2018	11	17

798
START : バスターズ	2018	11	18
DONE : バスターズ	2018	11	18

799
START : 麻雀覇道伝説 天牌外伝2	2018	11	18
**************************************************
麻雀覇道伝説 天牌外伝2 HAS NO RESULT
**************************************************
800
START : 麻雀覇道伝説　天牌外伝	2018	11	18
**************************************************
麻雀覇道伝説 天牌外伝 HAS NO RESULT
**************************************************
801
START : ボリショイ・バレエ in シネマ Season 2017-2018 「ロミオとジュリエット」	2018	11	18
DONE : ボリショイ・バレエ in シネマ Season 2017-2018 「ロミオとジュリエット」	2018	11	18

802
START : ボリショイ・バレエ in シネマ Season 2017-2018 「海賊」	2018	11	19
DONE : ボリショイ・バレエ in シネマ Season 2017-2018 「海賊」	2018	11	19

803
START : ボリショイ・バレエ in シネマ Season 2017-2018 「じゃじゃ馬ならし」	2018	11	20
DONE : ボリショイ・バレエ in シネマ Season 2017-2018 「じゃじゃ馬ならし」	2018	11	20

804
START : 書くが、まま	2018	11	21
DONE : 書くが、まま	201

DONE : マイ・サンシャイン	2018	12	15

898
START : メアリーの総て	2018	12	15
DONE : メアリーの総て	2018	12	15

899
START : モーターヘッド/クリーン・ユア・クロック	2018	12	15
DONE : モーターヘッド/クリーン・ユア・クロック	2018	12	15

900
START : 夢こそは、あなたの生きる未来	2018	12	15
DONE : 夢こそは、あなたの生きる未来	2018	12	15

901
START : 宵闇真珠	2018	12	15
DONE : 宵闇真珠	2018	12	15

902
START : レッド・ブレイド	2018	12	15
DONE : レッド・ブレイド	2018	12	15

903
START : 輪違屋糸里 京女たちの幕末	2018	12	15
DONE : 輪違屋糸里 京女たちの幕末	2018	12	15

904
START : ほっぷすてっぷじゃんぷッ！2	2018	12	16
DONE : ほっぷすてっぷじゃんぷッ！2	2018	12	16

905
START : アリー/ スター誕生	2018	12	21
DONE : アリー/ スター誕生	2018	12	21

906
START : シュガー・ラッシュ:オンライン	2018	12	21
DONE : シュガー・ラッシュ:オンライン	2018	12	21

907
START : ニセコイ	2018	12	21
DONE : ニセコイ	2018	12	21

908
START : 私は、マリア・カラス	2018	12	21
DONE : 私は、マリア・カラス	2018	12	21

909
START : いつか家族に	2018	12	22
DONE : いつか家族に	2018	12	22

910
START : 家へ帰ろう	2018	12	22
DONE : 家へ帰ろう	2018	12	22

911
START : 平成仮面ライダー20作記念 仮面ライダー平成ジェネレーションズ FOREVER	2018	12	22
DONE : 平成仮面ライダー20作記念 仮面ライダー平成ジェネレーションズ FOREVER	2018	12	22

912
START : からっぽ	